In [89]:
import sys
import os
import glob
import re

import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.head(3)

,id,賃料,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間


In [4]:
df.columns

Index(['id', '賃料', '所在地', 'アクセス', '間取り', '築年数', '方角', '面積', '所在階', 'バス・トイレ',
       'キッチン', '放送・通信', '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間'],
      dtype='object')

In [11]:
profile = pdp.ProfileReport(df)

In [12]:
profile.to_file(output_file="../processed_data/profiling_res.html")

In [60]:
df.columns = ['id','y','location','access','layout','age','direction','area','floor',
           'bath_toilet','kitchen','broadcast_com','facilities','parking','enviroment',
           'structure','contact_period']

In [61]:
df.to_csv('../processed_data/train.csv')

In [215]:
df_tar = df.loc[:,['id','age','area','floor','y']]

In [216]:
df_tar.head()

,id,age,area,floor,y
0,1,9年9ヶ月,20.01m2,1階／12階建,75000
1,2,44年10ヶ月,16.5m2,5階／10階建,76000
2,3,8年6ヶ月,22.05m2,12階／15階建,110000
3,4,29年4ヶ月,60.48m2,3階／4階建,150000
4,5,31年7ヶ月,39.66m2,1階／2階建,74000


### areaの前処理

- 単位m2を消去

In [218]:
def del_m2(a):
    return float(a[:-2])

In [219]:
df_tar['area'] = df_tar['area'].apply(del_m2)

In [220]:
df_tar.head(3)

,id,age,area,floor,y
0,1,9年9ヶ月,20.01,1階／12階建,75000
1,2,44年10ヶ月,16.50,5階／10階建,76000
2,3,8年6ヶ月,22.05,12階／15階建,110000


### ageの前処理

- 月単位に統一

In [221]:
hoge = df_tar.loc[1,'age']

In [222]:
hoge

'44年10ヶ月'

In [223]:
for i,s in enumerate(hoge):
# print(s)
    y,m = 0,0
    if s=="年":
        y = hoge[:i]
        tmp = i+1
    if s=="ヶ":
        m = hoge[tmp:i]

In [224]:
print(y,m)

0 0


In [225]:
def year_to_month(row):
    y,m = 0,0
    for i,s in enumerate(row):
        if s=="年":
            y = row[:i]
            tmp = i+1
        if s=="ヶ":
            m = row[tmp:i] 
    
    return int(y)*12 + int(m) 

In [226]:
df_tar['age'] = df_tar['age'].apply(year_to_month)

In [227]:
df_tar.head()

,id,age,area,floor,y
0,1,117,20.01,1階／12階建,75000
1,2,538,16.50,5階／10階建,76000
2,3,102,22.05,12階／15階建,110000
3,4,352,60.48,3階／4階建,150000
4,5,379,39.66,1階／2階建,74000


### floorの前処理

- floorに所在階
- 新たにmax_floorとして最上階を追加
- 地下は一旦無視

In [228]:
hoge = df_tar.loc[2,'floor']

In [229]:
hoge

'12階／15階建'

In [230]:
m = re.search(r'\d+階建', hoge)
m.group()[:-2]

'15'

In [231]:
def extract_floor(row):
    floor = re.search(r'\d+階／', row)
    
    try:
        floor = int(floor.group()[:-2])
    
    except:
        '''
        最上階が書かれていない場合の表記ゆれに対応
        '''
        floor = re.search(r'\d+階', row)
        floor = int(floor.group()[:-1])
    
    return floor

In [232]:
def max_floor_col(floor_col):
    tmp = []
    
    for f in floor_col:
        m = re.search(r'\d+階建', f)
        try:
            tmp.append(int(m.group()[:-2]))
            
        except:
            '''
            最上階についての記述がない場合は所在階を最上階と断定
            '''
            m = re.search(r'\d+階', f)
            tmp.append(int(m.group()[:-1]))
#             print(f)
            
    return tmp

In [233]:
df_tar['max_floor'] = max_floor_col(df_tar['floor'])

In [234]:
df_tar['floor'] = df_tar['floor'].apply(extract_floor)

In [235]:
df_tar.dtypes

id             int64
age            int64
area         float64
floor          int64
y              int64
max_floor      int64
dtype: object

In [236]:
df_tar['area']

0        20.01
1        16.50
2        22.05
3        60.48
4        39.66
         ...  
31465    37.90
31466    12.04
31467    60.00
31468    29.59
31469    25.00
Name: area, Length: 31470, dtype: float64

In [239]:
df_tar = df_tar.loc[:,['id','age','area','floor','max_floor','y']]

In [240]:
df_tar.head()

,id,age,area,floor,max_floor,y
0,1,117,20.01,1,12,75000
1,2,538,16.50,5,10,76000
2,3,102,22.05,12,15,110000
3,4,352,60.48,3,4,150000
4,5,379,39.66,1,2,74000


In [241]:
df_tar.to_csv('../processed_data/train_v1.csv',index=False)

In [242]:
df = pd.read_csv('../data/test.csv')

In [244]:
df.head()

,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,31471,東京都世田谷区深沢５丁目1-27,東急大井町線\t等々力駅\t徒歩15分\t\t東急田園都市線\t桜新町駅\t徒歩24分\t\...,2LDK,49年0ヶ月,南,50.22m2,8階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座／...,ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\tL字キッチン,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\tエレベーター\t公営水...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【コンビニ】 74m\t【スーパー】 215m\t【病院】 313m\t【小学校】 219m...,鉄骨造,NaN
1,31472,東京都目黒区八雲１丁目11-8,東急東横線\t都立大学駅\t徒歩6分\t\t東急東横線\t自由が丘駅\t徒歩17分\t\t東...,1R,0年2ヶ月,南東,20.88m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCSアンテナ／\tBSアンテナ／\tインターネット使用料無料,エアコン付\tシューズボックス／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水／...,"駐車場\t近隣\t34,000円(税込)\t距離200m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 128m\t【スーパー】 313m\t【コンビニ】 374m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間
2,31473,東京都豊島区池袋本町２丁目22-2,東武東上線\t北池袋駅\t徒歩10分\t\t埼京線\t板橋駅\t徒歩16分\t\t山手線\t...,1K,23年4ヶ月,南東,26.93m2,1階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t2面採光／\t室内洗濯...,"駐車場\t空有\t5,000円(税込)\t(敷金あり\t1ヶ月)\t駐輪場\t空有\tバイク...",【スーパー】 529m\t【スーパー】 810m\t【コンビニ】 316m\t【コンビニ】 ...,鉄骨造,2年間
3,31474,東京都杉並区和泉１丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩10分\t\t京王線\t代田橋駅\t徒歩8分...,1K,36年2ヶ月,南東,23.57m2,1階／2階建,シャワー,ガスコンロ／\t給湯,NaN,エアコン付\tシューズボックス／\t2面採光／\t室外洗濯機置場\t公営水道／\t下水／\t...,駐輪場\t空有\t0円,【スーパー】 286m\t【スーパー】 702m\t【コンビニ】 489m\t【コンビニ】 ...,木造,2年間
4,31475,東京都杉並区堀ノ内２丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩12分\t\t丸ノ内線(池袋－荻窪)\t新高...,2LDK,45年4ヶ月,南,50m2,4階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別,コンロ設置可（口数不明）／\t給湯,インターネット対応,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置...,"駐車場\t近隣\t22,600円\t距離177m",【コンビニ】 312m\t【コンビニ】 436m\t【スーパー】 902m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間


In [245]:
df.isnull().sum()

id           0
所在地          0
アクセス         0
間取り          0
築年数          0
方角        2808
面積           0
所在階          1
バス・トイレ     320
キッチン       635
放送・通信     3454
室内設備        73
駐車場       5356
周辺環境      9241
建物構造         0
契約期間      6981
dtype: int64